This is a worked through solution to [D&D.Sci (Easy Mode): On The Construction Of Impossible Structures](https://www.lesswrong.com/posts/Syfq6MwgdZhHg9vha/d-and-d-sci-easy-mode-on-the-construction-of-impossible)

In [5]:
]settings workdir ,[DYALOG]/Library/Conga   ⍝ Make sure we can find HttpCommand

* Command Execution Failed: cannot find the file "HttpCommand.dyalog" in any of
 the SALT USER folders

In [13]:
]box on -fns=on
]rows on -fold=3 -fns=on

Was ON -fns=off

Was OFF -fold=off -fns=off

In [7]:
]get ./D&D.Sci/data.csv

#.data

In [8]:
]get ./D&D.Sci/architect_proposals.txt

#.architect_proposals

In [21]:
⍴data

955 6

In [22]:
5↑data

Background of Architect Proposed Structure Type Required Construction Materials Characterization of Blueprints Is Structure Impossible? Cost of Structure 
 Apprenticed under T. Geisel Mansion Glass and Dreams Deceptively Ordinary No 29719 
 Apprenticed under T. Geisel Mechanism Silver and Steel Hastily Sketched No 79489 
 Apprenticed under M. Escher Tower Steel and Silver Obsessively Detailed No 100373 
 Self-Taught Mansion Wood and Dreams Obsessively Detailed Yes 736

In [23]:
≢architect_proposals

1870

In [32]:
800↑architect_proposals

Architect A, who was Self-Taught, presents Hastily Sketched blueprints proposing to build a Tower of Steel and Dreams
Architect B, who was Apprenticed under R. Penrose, presents Hastily Sketched blueprints proposing to build a Mansion of Steel and Nightmares
Architect C, who was Apprenticed under R. Penrose, presents Obsessively Detailed blueprints proposing to build a Library of Wood and Glass
Architect D, who was Apprenticed under B. Johnson, presents Obsessively Detailed blueprints proposing to build a Tower of Steel and Silver
Architect E, who was Apprenticed under P. Stamatin, presents Obsessively Detailed blueprints proposing to build a Mechanism of Silver and Wood
Architect F, who was Apprenticed under R. Penrose, presents Hastily Sketched blueprints proposing to build a Mansion of

By this, we can see we need a model that takes:

- background
- characteristics
- type
- materials

and predicts both the cost and whether the structure is impossible.

It looks like we ought to normalise the data according to each of our dimensions.

- bg: architect background
- typ: structure type
- mat: construction materials
- blu: characterisation of blueprints
- im: is structure impossible?
- cst: cost of structure

We can see that `mat` has overlap in the lists. Let's see whether our other fields are pure categories or if they have overlaps too.

In [26]:
10(⊂⍤?∘≢⌷⊢)data

Apprenticed under B. Johnson Library Steel and Silver Obsessively Detailed Yes 106717
 Self-Taught Library Wood and Nightmares Deceptively Ordinary Yes 406353
 Self-Taught Mansion Wood and Glass Hastily Sketched No 37946
 Apprenticed under P. Stamatin Tower Wood and Glass Deceptively Ordinary Yes 23309
 Apprenticed under B. Johnson Library Silver and Steel Obsessively Detailed Yes 69706
 Self-Taught Mechanism Steel and Glass Obsessively Detailed No 28890
 Apprenticed under P. Stamatin Mechanism Steel and Silver Hastily Sketched Yes 101614
 Self-Taught Mansion Wood and Dreams Hastily Sketched Yes 610
 Self-Taught Library Steel and Silver Obsessively Detailed No 94637
 Self-Taught Mansion Glass and Dreams Obsessively Detailed No 23124

It looks like they are all pure (can be encoded as indices into the unique items) except for materials which are two selected from those available.

What can we do to inspect these data? A hint is that anything more than spreadsheets is overkill...



In [29]:
d←1↓data

In [31]:
(↓⍉d[;⍳4]){⍺,≢⍵}⌸¨⊂d[;5]

Apprenticed under T. Geisel 149 Mansion 240 Glass and Dreams 44 Deceptively Ordinary 241 
 Apprenticed under M. Escher 83 Mechanism 257 Silver and Steel 60 Hastily Sketched 371 
 Self-Taught 340 Tower 326 Steel and Silver 54 Obsessively Detailed 342 
 Apprenticed under P. Stamatin 147 Library 131 Wood and Dreams 41 
 Apprenticed under R. Penrose 98 Glass and Silver 39 
 Apprenticed under B. Johnson 137 Wood and Nightmares 41 
 Silver and Wood 57 
 Wood and Silver 50 
 Glass and Steel 51 
 Steel and Wood 45 
 Wood and Glass 54 
 Silver and Glass 55 
 Steel and Nightmares 46 
 Glass and Nightmares 34 
 Silver and Nightmares 49 
 Silver and Dreams 29 
 Glass and Wood 52 
 Steel and Dreams 44 
 Steel and Glass 56 
 Wood and Steel 53

In [9]:
(h d)←{(1⌷⍵)(1↓⍵)}data   ⍝ Split header and data

In [18]:
(⌈/,⌊/)d[;h⍳⊂'Cost of Structure']

1041725 507

The range is quite large, so we will use the logarithm of costs in our analysis.

In [10]:
log_cost←⍟d[;h⍳⊂'Cost of Structure']

Now we look at creating indicator variables for our categorical variables.

We create indicator variables for each of the possible materials.

In [17]:
unique_mat←∪⊃,/pairs←(⊂1 3)∘⌷¨' '(≠⊆⊢)¨d[;h⍳⊂'Required Construction Materials']
materials←unique_mat∊⍤1↑pairs
10↑materials

1 1 0 0 0 0
0 0 1 1 0 0
0 0 1 1 0 0
0 1 0 0 1 0
1 0 1 0 0 0
0 1 0 0 1 0
0 0 0 0 1 1
0 0 1 1 0 0
0 0 1 1 0 0
1 1 0 0 0 0

In [25]:
h

┌───────────────────────┬───────────────────────┬───────────────────────────────┬──────────────────────────────┬────────────────────────┬─────────────────┐
│Background of Architect│Proposed Structure Type│Required Construction Materials│Characterization of Blueprints│Is Structure Impossible?│Cost of Structure│
└───────────────────────┴───────────────────────┴───────────────────────────────┴──────────────────────────────┴────────────────────────┴─────────────────┘

In [29]:
+/'Yes'∘≡¨d[;h⍳⊂'Is Structure Impossible?']                           ⍝ How many impossible structures in our data?
≢∪d[;1 2 3 4]⌿⍨'Yes'∘≡¨d[;h⍳⊂'Is Structure Impossible?']             ⍝ Are there particular combinations that lead to impossible structures?
≢∪(materials,d[;1 2 4])⌿⍨'Yes'∘≡¨d[;h⍳⊂'Is Structure Impossible?']   ⍝ What about if materials are considered separately?

434

315

263

Let's get some percentages of our categorical variables against whether they create impossible structures.

In [36]:
n←≢d
d[;h⍳⊂'Background of Architect'],∘(+⌿÷≢)⌸'Yes'∘≡¨d[;h⍳⊂'Is Structure Impossible?']

┌─────────────────────────────┬────────────┐
│Apprenticed under T. Geisel │0 │
├─────────────────────────────┼────────────┤
│Apprenticed under M. Escher │0 │
├─────────────────────────────┼────────────┤
│Self-Taught │0.4411764706│
├─────────────────────────────┼────────────┤
│Apprenticed under P. Stamatin│1 │
├─────────────────────────────┼────────────┤
│Apprenticed under R. Penrose │0 │
├─────────────────────────────┼────────────┤
│Apprenticed under B. Johnson │1 │
└─────────────────────────────┴────────────┘

So our impossible structures were created by apprentices of P. Stamatin and B. Johnson, and some self-taught architects. Under what conditions do self-taught architects create impossible structures?

In [46]:
stm←{(⊂⍒⊢/⍵)⌷⍵}(materials,'Self-Taught'∘≡¨d[;h⍳⊂'Background of Architect']),∘(+⌿÷≢)⌸'Yes'∘≡¨d[;h⍳⊂'Is Structure Impossible?']
(unique_mat,'Self-Taught' 'pct impossible')⍪stm

┌─────┬──────┬──────┬─────┬────┬──────────┬───────────┬──────────────┐
│Glass│Dreams│Silver│Steel│Wood│Nightmares│Self-Taught│pct impossible│
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┼──────────────┤
│0 │1 │0 │0 │1 │0 │1 │0.8 │
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┼──────────────┤
│0 │1 │1 │0 │0 │0 │1 │0.8 │
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┼──────────────┤
│1 │1 │0 │0 │0 │0 │0 │0.652173913 │
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┼──────────────┤
│0 │1 │0 │1 │0 │0 │0 │0.5925925926 │
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┼──────────────┤
│1 │0 │0 │0 │0 │1 │0 │0.5789473684 │
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┼──────────────┤
│1 │0 │0 │0 │0 │1 │1 │0.5333333333 │
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┼──────────────┤
│1 │0 │0 │0 │1 │0 │1 │0.5277777778 │
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┼──────────────┤
······································································
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┼──────────────┤
│0 │0 │0 │0 │1 │1 │0 │0.2222222222 │
└─────┴──────┴──────┴─────┴────┴──────────┴───────────┴──────────────┘

It looks like self-taught architects who use dreams are highly likely to create impossible structures.

We suspect structure type and materials might contribute to cost. Let's investigate that relationship.

In [50]:
⍝ Mean cost by structure type
d[;h⍳⊂'Proposed Structure Type'],∘(+⌿÷≢)⌸d[;h⍳⊂'Cost of Structure']

⍝ Mean cost for each material (using your materials matrix)
(unique_mat,⊆'avg cost')⍪{(⊂⍒⊢/⍵)⌷⍵}materials,∘(+⌿÷≢)⌸d[;h⍳⊂'Cost of Structure']

┌─────────┬───────────┐
│Mansion │148090.6875│
├─────────┼───────────┤
│Mechanism│141000.035 │
├─────────┼───────────┤
│Tower │145767.0429│
├─────────┼───────────┤
│Library │194573.4351│
└─────────┴───────────┘

┌─────┬──────┬──────┬─────┬────┬──────────┬───────────┐
│Glass│Dreams│Silver│Steel│Wood│Nightmares│avg cost │
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┤
│0 │0 │1 │0 │0 │1 │675334.4082│
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┤
│0 │0 │0 │0 │1 │1 │613753.6341│
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┤
│0 │0 │0 │1 │0 │1 │603472.2609│
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┤
│1 │0 │0 │0 │0 │1 │597313.1471│
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┤
│1 │0 │1 │0 │0 │0 │104279.8191│
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┤
│0 │0 │1 │1 │0 │0 │78917.96491│
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┤
│0 │0 │1 │0 │1 │0 │74961.01869│
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┤
·······················································
├─────┼──────┼──────┼─────┼────┼──────────┼───────────┤
│0 │1 │0 │0 │1 │0 │757.4634146│
└─────┴──────┴──────┴─────┴────┴──────────┴───────────┘

In [62]:
{(⊂⍒⊢/⍵)⌷⍵}(↑pairs),∘(+⌿÷≢)⌸d[;h⍳⊂'Cost of Structure']

┌──────┬──────────┬───────────┐
│Silver│Nightmares│675334.4082│
├──────┼──────────┼───────────┤
│Wood │Nightmares│613753.6341│
├──────┼──────────┼───────────┤
│Steel │Nightmares│603472.2609│
├──────┼──────────┼───────────┤
│Glass │Nightmares│597313.1471│
├──────┼──────────┼───────────┤
│Silver│Glass │105048.4727│
├──────┼──────────┼───────────┤
│Glass │Silver │103195.8205│
├──────┼──────────┼───────────┤
│Silver│Steel │80270.03333│
├──────┼──────────┼───────────┤
│Steel │Silver │77415.66667│
├──────┼──────────┼───────────┤
·······························
├──────┼──────────┼───────────┤
│Wood │Dreams │757.4634146│
└──────┴──────────┴───────────┘

In [63]:
materials[;unique_mat⍳⊂'Nightmares'],∘(+⌿÷≢)⌸d[;h⍳⊂'Cost of Structure']

0 49061.59949
1 625433.2706

So we can see that nightmares are significantly more expensive than any other kind of material.

We can certainly attempt a regression on materials vs $\log(cost)$.

In [56]:
mat_cost←materials,∘(+⌿÷≢)⌸⍟d[;h⍳⊂'Cost of Structure']
⎕←coeff←(⊢/mat_cost)⌹¯1↓⍤1⊢mat_cost

5.705262932 3.888690935 6.378316762 4.834001612 4.268226354 7.949478661

In [58]:
*0 0 1 1 0 0+.×coeff
*0 1 0 0 1 0+.×coeff

74036.86137

3487.419307

So those most likely to create impossible structures are apprentices of P. Stamatin and B. Johnson, or self-taught architects who use dreams. We want to avoid proposals to build with nightmares because they are expensive.

In [65]:
300↑architect_proposals

Architect A, who was Self-Taught, presents Hastily Sketched blueprints proposing to build a Tower of Steel and Dreams
Architect B, who was Apprenticed under R. Penrose, presents Hastily Sketched blueprints proposing to build a Mansion of Steel and Nightmares
Architect C, who was Apprenticed under R.

The pattern is:
```
Architect N, who was [background], presents [blueprint] proposing to build a [type] of [materials]
```

In [77]:
ap←⎕CSV architect_proposals 'S' ⍬
ap[;2]↓⍨←8   ⍝ Drop "who was "
ap[;3]↓⍨←9   ⍝ Drop "presents "
n←≢'blueprints proposing to build a '
ap[;3]↓⍨←n+' blueprints'∘(⍸⍷)¨ap[;3]
⎕←proposals←ap[;1 2],↑(⊂1 3 5)∘⌷¨' '(≠⊆⊢)¨ap[;3]

┌───────────┬─────────────────────────────┬─────────┬──────┬──────────┐
│Architect A│Self-Taught │Tower │Steel │Dreams │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect B│Apprenticed under R. Penrose │Mansion │Steel │Nightmares│
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect C│Apprenticed under R. Penrose │Library │Wood │Glass │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect D│Apprenticed under B. Johnson │Tower │Steel │Silver │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect E│Apprenticed under P. Stamatin│Mechanism│Silver│Wood │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect F│Apprenticed under R. Penrose │Mansion │Silver│Dreams │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect G│Apprenticed under P. Stamatin│Mechanism│Glass │Nightmares│
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect H│Apprenticed under B. Johnson │Tower │Silver│Steel │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
·······································································
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect N│Self-Taught │Mechanism│Steel │Wood │
└───────────┴─────────────────────────────┴─────────┴──────┴──────────┘

In [80]:
imp←proposals[;2]∊'Apprenticed under P. Stamatin' 'Apprenticed under B. Johnson'
proposals[⍒imp;]

┌───────────┬─────────────────────────────┬─────────┬──────┬──────────┐
│Architect D│Apprenticed under B. Johnson │Tower │Steel │Silver │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect E│Apprenticed under P. Stamatin│Mechanism│Silver│Wood │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect G│Apprenticed under P. Stamatin│Mechanism│Glass │Nightmares│
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect H│Apprenticed under B. Johnson │Tower │Silver│Steel │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect K│Apprenticed under P. Stamatin│Library │Glass │Dreams │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect A│Self-Taught │Tower │Steel │Dreams │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect B│Apprenticed under R. Penrose │Mansion │Steel │Nightmares│
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect C│Apprenticed under R. Penrose │Library │Wood │Glass │
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
·······································································
├───────────┼─────────────────────────────┼─────────┼──────┼──────────┤
│Architect N│Self-Taught │Mechanism│Steel │Wood │
└───────────┴─────────────────────────────┴─────────┴──────┴──────────┘

There are 5 proposals guaranteed to create impossible structures, but one intends to use nightmares. The four proposals to accept are D, E, H and K.